# La QFT y sumadores cuánticos



Mario Quiñones Pérez

# Forest

In [1]:
from pyquil import get_qc, Program
from pyquil.api import QVMConnection
from pyquil.gates import *
from pyquil import latex
from pyquil.api import local_forest_runtime
from pyquil import get_qc, Program

Para la creación de circuitos se usará la función latex.to_latex(Program()), pero al crear esta función un string que pasar a TexWorks, solo se usará una vez como ejemplo para no ensuciar la memoria.

## Algoritmo de Deutsch

El problema trata de que se nos da un circuito (Oráculo) que implementa una función booleana de un bit y hay que determinar si la función es constante(mismo valor para todas las entradas) o balanceada (1 en una entrada y 0 en la otra).


En el escenario clásico, necesitaríamos consultar esta caja negra dos veces, para calcular ambos valores de la función midiendo f(0) y luego f(1).


En el escenario cuántico, podemos hacerlo con solo una evaluación, en superposición, y utilizando el entrelazamiento y al final interferencia.


Este circuito calcula, de forma reversible (algo que se debe cumplir para todo circuito cuántico), una cierta función f (en este caso, de una sola entrada)

    Si la función es constante, mediremos “0”
    Si la función es balanceada mediremos “1”

El algoritmo de Deutsch explota un fenómeno de interferencia similar al encontrado en algunos experimentos físicos. Por ejemplo el experimento de la doble rendija o Interferómetro de Mach-Zehnder.

Creamos los siguientes oraculos para la prueba de el algoritmo de Deutsch

In [2]:
def NOT(p):
    p += X(1)

In [3]:
def CNOTs(p):
    p += CNOT(0,1)

In [4]:
def NOTCNOT(p):
    p += X(0)
    p += CNOT(0,1)
    p += X(0)

Creamos el circuito para la comprobación de un oráculo, en este añadimos un qubit auxiliar que inicializamos a |1> y pondremos ambos eb superposición pasandolos al estado |+> y |-> con las puertas Hadamard. Pasaremos estos qubits por el oráculo y tras este paso deshacemos la superposición y medimos el qubit q0 que es el que nos interesa

Este será la única parte de la memoria en la que se mostrará el código generado por la función latex.to_latex(p).

In [5]:
p = Program()
ro = p.declare('ro', 'BIT', 1)

p += X(1)
p += H(0)
p += H(1)

p += FENCE()

NOTCNOT(p)

p += FENCE()

p += H(0)
p += MEASURE(0, ro[0])

print(p)

\documentclass[convert={density=300,outext=.png}]{standalone}
\usepackage[margin=1in]{geometry}
\usepackage{tikz}
\usetikzlibrary{quantikz}
\begin{document}
\begin{tikzcd}
\lstick{\ket{q_{0}}} & \gate{H} & \gate{X} & \ctrl{1} & \gate{X} & \gate{H} & \meter{} & \qw \\
\lstick{\ket{q_{1}}} & \gate{X} & \gate{H} & \targ{} & \qw & \qw & \qw & \qw
\end{tikzcd}
\end{document}
DECLARE ro BIT[1]
X 1
H 0
H 1
FENCE
X 0
CNOT 0 1
X 0
FENCE
H 0
MEASURE 0 ro[0]



![1.png](1.png)

### Simulaciones

Vemos que para la simulación con un oraculo balanceado como "NOTCNOT" la salida sale 1 mientras que para un oraculo constante como la identidad la salida sale siempre 0

In [11]:
p = Program()
ro = p.declare('ro', 'BIT', 1)

p += X(1)
p += H(0)
p += H(1)


p += H(0)
p += MEASURE(0, ro[0])


qc = get_qc('2q-qvm')
executable = qc.compile(p)
result = qc.run(executable)
   
print(result)

[[0]]


In [12]:
p = Program()
ro = p.declare('ro', 'BIT', 1)

p += X(1)
p += H(0)
p += H(1)


NOTCNOT(p)


p += H(0)
p += MEASURE(0, ro[0])


qc = get_qc('2q-qvm')
executable = qc.compile(p)
result = qc.run(executable)
   
print(result)

[[1]]


## Algoritmo de Deutsch-Jozsa

El algoritmo Deutsch-Jozsa resuelve un tipo de problema llamados de consulta, oráculo o de promesa.

Se nos da una función booleana f de más de un parámetro y se nos promete que es constante o balanceado (0 para la mitad de las entradas y 1 para el resto).  Tenemos que averiguar cual de las dos es llamando a la función el menor número de veces posible.

Mientras que con un algoritmo determinista clásico necesitamos (en el peor caso) 2(n-1) + 1 llamadas a f, con el algoritmo cuántico Deutsch-Jozsa es suficiente evaluarlo una vez.



1. Creamos el estado | (0^n) 1 >
2. Usamos H para crear superposición
3. Aplicamos el oráculo
4. Aplicamos de nuevo H a los n primeros qbits
5. Medimos los n primeros qbits

Si la función es constante, obtendremos 0 y si es balanceada, obtendremos una cadena distinta. 


Utilizaremos la función prepare para preparar un circuito a partir de una cantidad de qubits cualesquiera "n" en un circuito "p". Se añadirá un qubit auxiliar inicializado a uno y se pondrán todos en superposición.

In [8]:
def prepare(p, n):
    p += X(n)

    for i in range(n + 1):
        p += H(i)
        
    return p

Utilizaremos la función end quitar los n primeros qubits del circuito de superposición y añadir mediciones a todos los qubits menos al último.

In [9]:
def end(p, n, ro):

    for i in range(n):
        p += H(i)
        
    lista = list(range(n))
        
    for i, q in enumerate(lista):
        p += MEASURE(q, ro[i])

He creado una serie de oráculos basándose en ejercicios dados y aquellos ya mostrados en clase.

In [10]:
def ejercicio1(p, n):
    for i in range(n):
        p += CNOT(i, n)
        p += X(i)
        p += CNOT(i, n)

In [11]:
def ejercicio2(p, n):
        p += CNOT(2, n)
        p += CNOT(1, n)
        p += CCNOT(0, 1, n)

In [12]:
def cnots(p, n):
    for i in range(n):
        p += CNOT(i, n)

In [13]:
def Rcnots(p, n):
    for i in range(n):
        p += CNOT((n-1) - i,n)

In [14]:
n = 3
p = Program()
ro = p.declare('ro', 'BIT', n)

prepare(p, n)

p += FENCE()

ejercicio1(p, n)

p += FENCE()

end(p, n, ro)

print(p)

DECLARE ro BIT[3]
X 3
H 0
H 1
H 2
H 3
FENCE
CNOT 0 3
X 0
CNOT 0 3
CNOT 1 3
X 1
CNOT 1 3
CNOT 2 3
X 2
CNOT 2 3
FENCE
H 0
H 1
H 2
MEASURE 0 ro[0]
MEASURE 1 ro[1]
MEASURE 2 ro[2]



![2.png](2.png)

In [15]:
n = 3
p = Program()
ro = p.declare('ro', 'BIT', n)

prepare(p, n)

p += FENCE()

ejercicio2(p, n)

p += FENCE()

end(p, n, ro)

print(p)

DECLARE ro BIT[3]
X 3
H 0
H 1
H 2
H 3
FENCE
CNOT 2 3
CNOT 1 3
CCNOT 0 1 3
FENCE
H 0
H 1
H 2
MEASURE 0 ro[0]
MEASURE 1 ro[1]
MEASURE 2 ro[2]



![3.png](3.png)

### Simulaciones

Como podemos comprobar en las siguientes ejecuciones, las funciones cuya salida para todos los qubits es siempre |0> son constantes, mientras que aquellas cuya salida es distinta de este valor son balanceadas.

In [16]:
n = 4
p = Program()
ro = p.declare('ro', 'BIT', n)

prepare(p, n)

ejercicio1(p, n)

end(p, n, ro)

qc = get_qc('5q-qvm')
executable = qc.compile(p)
result = qc.run(executable)
    

print(result)

[[0 0 0 0]]


In [17]:
n = 4
p = Program()
ro = p.declare('ro', 'BIT', n)

prepare(p, n)

ejercicio2(p, n)

end(p, n, ro)

qc = get_qc('5q-qvm')
executable = qc.compile(p)
result = qc.run(executable)
    

print(result)

[[1 0 1 0]]


In [18]:
n = 4
p = Program()
ro = p.declare('ro', 'BIT', n)

prepare(p, n)

cnots(p, n)

end(p, n, ro)

qc = get_qc('5q-qvm')
executable = qc.compile(p)
result = qc.run(executable)
    

print(result)

[[1 1 1 1]]


In [27]:
n = 4
p = Program()
ro = p.declare('ro', 'BIT', n)

prepare(p, n)

Rcnots(p, n)

end(p, n, ro)

qc = get_qc('5q-qvm')
executable = qc.compile(p)
result = qc.run(executable)
    

print(result)

[[1 1 1 1]]
